# 🔍 RAG 챗봇의 품질을 높이는 고급 전략

---

## 🎯 강의 목표

- 단순히 "문서를 불러와 답하는" 챗봇이 아닌,  
  **신뢰도 높은, 실제 현업에 쓸 수 있는 고품질 RAG 챗봇**을 만드는 데 필요한 전략을 익힙니다.

- 검색 품질과 응답 품질, 이 두 가지 모두에 초점을 맞춥니다.

---

## 🧠 1. Re-ranking: 검색 정확도 향상을 위한 두 번째 단계

### ✅ RAG의 한계
- 벡터 유사도 기반 검색은 **“대충 비슷한 문장”**을 찾지만,  
  반드시 질문과 **정확히 관련된 문장**은 아닐 수 있음

> 예: "퇴직 절차"를 묻는 질문에 "채용 절차" 문서가 상위에 나올 수 있음 (유사하지만 부정확)

---

### 🧩 해결책: Re-ranking 기법

- ✅ **Dense Retriever + Cross Encoder 구조**
  1. **1차 검색**: FAISS, Chroma 등으로 유사 문서 Top-k 검색
  2. **2차 평가**: LLM 또는 Cross-Encoder가 질문과 문서의 semantic match를 재점수화

---

### 🔧 사용 가능한 Re-ranker

| 모델명 | 특징 |
|--------|------|
| **Cohere Rerank** | 질문-문서 유사도 재정렬, API 기반 |
| **BGE-Reranker** | open-source, huggingface 기반 |
| **Cross-Encoder (sentence-transformers)** | 정확도 높지만 느림 |
| **LLM 기반 ScoreChain** | OpenAI function call로 구현 가능 |

---

### 💡 예시 코드 (Cohere Rerank)

```python
from cohere import Client
client = Client("API_KEY")

results = client.rerank(
    query="퇴직 절차",
    documents=["채용 절차 안내", "퇴직 시 준비 서류", "복지 혜택"],
    top_n=2
)

for r in results:
    print(r.document, r.relevance_score)
````

---

## ✂️ 2. Chunking 전략: 어떻게 자르느냐가 품질을 좌우한다

### ✅ 기본 전략: 문서를 나눠야 임베딩이 가능함

| 전략                 | 설명        | 장단점            |
| ------------------ | --------- | -------------- |
| **Fixed Chunk**    | 일정 길이로 자름 | 간단하지만 문맥 끊김 가능 |
| **Sliding Window** | 겹치며 자름    | 문맥 보존 ↑, 계산량 ↑ |
| **Sentence-aware** | 문장 단위로 자름 | 품질 ↑, 구현 복잡 ↑  |

---

### 📊 실험 예시: Chunk 길이와 검색 결과 비교

* Chunk 1 (No Overlap):

  > “...우리는 세계 평화를 추구하며...”
* Chunk 2 (Overlap=50%):

  > “...세계 평화를 추구하며 인류 공영을...”

→ **질문**: “헌법 전문에 나오는 평화 관련 내용은?”
→ Overlap이 있을 때 더 정확한 문맥이 검색됨

---

## 🧾 3. 프롬프트 설계: 답변 품질을 바꾸는 작은 변화

### 🤖 단순 프롬프트의 한계

```text
"다음 문서를 기반으로 질문에 답하세요: {context} Q: {query}"
```

* 너무 단조롭고, **LLM이 근거 문서 활용을 무시**할 수 있음
* 정답 같지만 문서와 관련 없는 "hallucination 답변"이 발생 가능

---

### ✅ 고급 프롬프트 전략

| 전략            | 예시                           |
| ------------- | ---------------------------- |
| **근거 강제**     | “반드시 문서에 기반하여 근거를 인용하세요.”    |
| **답변 형식 제시**  | “답변은 3줄 이내 요약, 마지막 줄에 출처 포함” |
| **출처 태깅**     | "출처: {document\_id}" 자동 삽입   |
| **사용자 친화 응답** | “질문자에게 직접 설명하듯 답해주세요.”       |

---

### 💬 예시 프롬프트 개선 전후

#### Before:

```text
{context}
Q: 퇴직 절차는 어떻게 되나요?
```

#### After:

```text
다음 문서에서 퇴직 절차에 관한 내용을 찾아 요약해주세요.
답변은 문서 내용을 기반으로 3줄 이내로 작성하고, 출처 문장을 인용하세요.

{context}

Q: 퇴직 절차는 어떻게 되나요?
```

---

## 🧩 마무리 요약

| 전략               | 요약                        |
| ---------------- | ------------------------- |
| **Re-ranking**   | 유사한 문서 중 실제로 관련 있는 문서만 선택 |
| **Chunking 최적화** | 문맥 손실을 줄이는 것이 핵심          |
| **프롬프트 설계**      | 신뢰도 높은 답변을 유도하는 구조 필요     |

---

## 🎯 다음 강의 예고

다음 강의에서는
"운영 환경에서 RAG 챗봇을 어떻게 관리하고 최적화할 수 있는지"에 대해 다룰 예정입니다.